In [12]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse
os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")


from config import cfg
from logger import logger
import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
from file_io import load_urls, dump_urls
import joblib as jl
import url_forest
reload(url_forest)

<module 'url_forest' from '/home/sparrow/sparrow/malicious_url_cluster/url_forest.pyc'>

In [13]:
df_domain_url = pd.read_csv("../../data/EXP_domain/df_res_url.csv")
df_domain_id  = pd.read_csv("../../data/EXP_domain/df_res_sha256.csv")
df_domain_fp = pd.concat([df_domain_url.loc[df_domain_url.sha256==_]  \
                          for _ in list(df_domain_id.loc[df_domain_id.positives == 0].sha256)])
df_domain_tp = pd.concat([df_domain_url.loc[df_domain_url.sha256==_]  \
                          for _ in list(df_domain_id.loc[df_domain_id.positives >0].sha256)])

In [15]:
df_url_url = pd.read_csv("../../data/EXP_url/df_res_url.csv")
df_url_id  = pd.read_csv("../../data/EXP_url/df_res_sha256.csv")
df_url_fp  = pd.concat([df_url_url.loc[df_url_url.sha256==_]  \
                          for _ in list(df_url_id.loc[df_url_id.positives == 0].sha256)])
df_url_tp  = pd.concat([df_url_url.loc[df_url_url.sha256==_]  \
                          for _ in list(df_url_id.loc[df_url_id.positives >0].sha256)])

In [16]:
print "url regex detection\t%d\turl detection\t%d" %(len(set(df_url_url.sha256)), len(set(df_url_url.url)))
print "url regex fp\t\t%d\turl fp\t\t%d" %(len(set(df_url_fp.sha256)), len(set(df_url_fp.url)))


url regex detection	69	url detection	1145
url regex fp		29	url fp		87


In [17]:
print "domain regex detection\t%d\tdomain detection\t%d" %(len(set(df_domain_url.sha256)), len(set(df_domain_url.url)))
print "domain regex fp\t\t%d\tdomain fp\t\t%d" %(len(set(df_domain_fp.sha256)), len(set(df_domain_fp.url)))

domain regex detection	71	domain detection	3368
domain regex fp		31	domain fp		31


In [19]:
print "regex total detection\t%d\turl total detection\t%d" %(len(set(df_domain_url.sha256) | set(df_url_url.sha256)),
                                                             len(set(df_domain_url.url)|set(df_url_url.url)))
print "regex total fp\t\t%d\turl total fp\t\t%d" %(len(set(df_domain_fp.sha256) | set(df_url_fp.sha256)),
                                                             len(set(df_domain_fp.url)|set(df_url_fp.url)))

regex total detection	136	url total detection	3506
regex total fp		60	url total fp		118


In [100]:
safe_urls = load_urls("../../data/sangfor/safe_0419_mod.csv", csv = True)
safe_url_domain, safe_url_path = preprocess.make_url_split(safe_urls)
print len(safe_url_domain)

mal_urls = load_urls("../../data/EXP_domain/malware.csv", csv = True)
mal_url_domain, mal_url_path = preprocess.make_url_split(mal_urls)
print len(mal_url_domain)

2018-04-25 12:55:30 file_io.py [line:52] DEBUG	URLs Count:	1454804


466910


2018-04-25 12:55:56 file_io.py [line:52] DEBUG	URLs Count:	1303296


453097


In [101]:
safe_domain_forest = url_forest.build_domain_forest(safe_url_domain)
mal_domain_forest = url_forest.build_domain_forest(mal_url_domain)
jl.dump(safe_domain_forest, "../../data/url_forest/safe_domain_forest_mod.jl.z")
jl.dump(mal_domain_forest, "../../data/url_forest/mal_domain_forest.jl.z")

building tree index:0
building tree index:1
building tree index:2
building tree index:3
building tree index:4
building tree index:5
building tree index:6
building tree index:7
building tree index:8
building tree index:9
building tree index:0
building tree index:1
building tree index:2
building tree index:3
building tree index:4
building tree index:5
building tree index:6
building tree index:7
building tree index:8
building tree index:9


['../../data/url_forest/mal_domain_forest.jl.z']

In [2]:
safe_domain_forest = jl.load("../../data/url_forest/safe_domain_forest_mod.jl.z")
mal_domain_forest  = jl.load("../../data/url_forest/mal_domain_forest.jl.z")

In [116]:
domain_reputation_thresh = 0.8

In [114]:
df = url_forest.analyze_domain_batch(list(df_domain_url.url), safe_domain_forest, mal_domain_forest, verbose = False)
df_publish = df.loc[df.domain_reputation>domain_reputation_thresh]

sel = [i in set(df_publish.url) for i in df_domain_url.url]
df_domain_url_p = df_domain_url.loc[sel]
sel = [i in set(df_publish.url) for i in df_domain_fp.url]
df_domain_fp_p = df_domain_fp.loc[sel]

print "domain regex detection\t%d\tdomain detection\t%d" %(len(set(df_domain_url_p.sha256)), len(set(df_domain_url_p.url)))
print "domain regex fp\t\t%d\tdomain fp\t\t%d" %(len(set(df_domain_fp_p.sha256)), len(set(df_domain_fp_p.url)))

domain regex detection	20	domain detection	3315
domain regex fp		1	domain fp		1


In [117]:
df = url_forest.analyze_domain_batch(list(df_url_url.url), safe_domain_forest, mal_domain_forest, verbose = False)
df_publish = df.loc[df.domain_reputation>domain_reputation_thresh]

sel = [i in set(df_publish.url) for i in df_url_url.url]
df_url_url_p = df_url_url.loc[sel]
sel = [i in set(df_publish.url) for i in df_url_fp.url]
df_url_fp_p = df_url_fp.loc[sel]

print "url regex detection\t%d\turl detection\t%d" %(len(set(df_url_url_p.sha256)), len(set(df_url_url_p.url)))
print "url regex fp\t\t%d\turl fp\t\t%d" %(len(set(df_url_fp_p.sha256)), len(set(df_url_fp_p.url)))

url regex detection	29	url detection	1056
url regex fp		2	url fp		18


In [54]:
print "regex total detection\t%d\turl total detection\t%d" %(len(set(df_domain_url_p.sha256) | set(df_url_url_p.sha256)),
                                                             len(set(df_domain_url_p.url)|set(df_url_url_p.url)))
print "regex total fp\t\t%d\turl total fp\t\t%d" %(len(set(df_domain_fp_p.sha256) | set(df_url_fp_p.sha256)),
                                                             len(set(df_domain_fp_p.url)|set(df_url_fp_p.url)))

regex total detection	45	url total detection	3329
regex total fp		3	url total fp		19


In [120]:
"""domain reputation analyze"""
url_forest.analyze_domain(safe_domain_forest, mal_domain_forest, domain="download.g", verbose = True)

1 1


0.9900990099009901

In [119]:
df_publish.loc[df_publish.domain_reputation<1]

,domain_reputation,url
40,0.990099,http://download.glzip.cn/n/install/version/v2....
112,0.990099,http://download.glzip.cn/n/install/version/v2....
171,0.990099,http://download.glzip.cn/n/install/version/v2....
181,0.990099,http://download.glzip.cn/n/install/version/v2....
269,0.990099,http://download.glzip.cn/n/install/version/v2....
325,0.990099,http://client-jupai.api.onfeng.com/temp/taskSe...
330,0.990099,http://xixi.conglebabyswim.com/LO/LO_dz206?0gp...
582,0.990099,http://client-jupai.api.onfeng.com/temp/Runtim...
610,0.990099,http://download.glzip.cn/n/install/version/v2....
629,0.990099,http://download.glzip.cn/n/install/version/v2....
